# Sentiment Analysis

## IMDB Review

In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv(filename, sep = '\t', quoting = 3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

### Preprocessing

In [6]:
df['review'] = df.review.str.replace('<br />', ' ')

In [7]:
import re
df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]', ' ', x))

In [9]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

### 학습 및 테스트 데이터 셋 분리

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, test_size = 0.2,
    stratify = df.sentiment, random_state = 2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

### 텍스트 변환

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#### Count Vectorizer, N-Gram Range : 1 ~ 1

##### 잘못된 방법

In [12]:
cvect1 = CountVectorizer(stop_words = 'english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)

In [13]:
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 37187))

##### 적절한 방법

In [14]:
cvect1 = CountVectorizer(stop_words = 'english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 66971))

#### Count Vectorizer, N-Gram Range : 1 ~ 2

In [21]:
cvect2 = CountVectorizer(stop_words = 'english', ngram_range = (1, 2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((20000, 1461350), (5000, 1461350))

In [22]:
lr2 = LogisticRegression(max_iter = 500)
%time lr2.fit(X_train_cv2, y_train)

CPU times: user 1min 14s, sys: 40.6 s, total: 1min 54s
Wall time: 1min 4s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
pred2 = lr2.predict(X_test_cv2)

In [24]:
score2 = accuracy_score(y_test, pred2)
score2

0.8914

#### TF-IDF, N-Gram Range : 1 ~ 1

In [25]:
tvect1 = TfidfVectorizer(stop_words = 'english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((20000, 66971), (5000, 66971))

In [26]:
lr3 = LogisticRegression(max_iter = 500)
%time lr3.fit(X_train_tv1, y_train)

CPU times: user 2.44 s, sys: 2.88 s, total: 5.32 s
Wall time: 2.77 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
pred3 = lr3.predict(X_test_tv1)

In [28]:
score3 = accuracy_score(y_test, pred3)
score3

0.893

#### TF-IDF, N-Gram Range : 1 ~ 2

In [29]:
tvect2 = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((20000, 1461350), (5000, 1461350))

In [30]:
lr4 = LogisticRegression(max_iter = 500)
%time lr4.fit(X_train_tv2, y_train)

CPU times: user 21.9 s, sys: 12.1 s, total: 34 s
Wall time: 18.9 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
pred4 = lr4.predict(X_test_tv2)

In [32]:
score4 = accuracy_score(y_test, pred4)
score4

0.883

### 로지스틱 회귀 모델 학습(훈련)

In [15]:
from sklearn.linear_model import LogisticRegression

#### 객체 생성 및 훈련 - 훈련 데이터의 X, y

In [16]:
lr1 = LogisticRegression(max_iter = 500)
%time lr1.fit(X_train_cv1, y_train)

CPU times: user 8.3 s, sys: 9.63 s, total: 17.9 s
Wall time: 10.9 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### 테스트 데이터에 대해서 예측 - 테스트 데이터의 X

In [17]:
pred1 = lr1.predict(X_test_cv1)

### 평가

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
score1 = accuracy_score(y_test, pred1)
score1

0.8832